# MLOPT Knapsack Example

In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import logging

import mlopt
from mlopt.sampling import uniform_sphere_sample
from mlopt.learners.pytorch.pytorch import PyTorchNeuralNet
from mlopt.utils import n_features, pandas2array

## Generate problem data

In [2]:
np.random.seed(1)  # Reset random seed for reproducibility

# Variable
n = 10
x = cp.Variable(n, integer=True)

# Cost
c = np.random.rand(n)

# Weights
a = cp.Parameter(n, nonneg=True, name='a')
x_u = cp.Parameter(n, nonneg=True, name='x_u')
b = 0.5 * n

## Create optimizer object

In [3]:
# Problem
cost = - c * x
constraints = [a * x <= b,
               0 <= x, x <= x_u]


# Define optimizer
# If you just want to remove too many messages
# change INFO to WARNING
m = mlopt.Optimizer(cp.Minimize(cost), constraints,
                    log_level=logging.INFO)

## Define training and testing parameters

In [6]:
# Average request
theta_bar = 2 * np.ones(2 * n)
radius = 1.0


def sample(theta_bar, radius, n=100):

    # Sample points from multivariate ball
    ndim = int(len(theta_bar)/2)
    X_a = uniform_sphere_sample(theta_bar[:ndim], radius, n=n)
    X_u = uniform_sphere_sample(theta_bar[ndim:], radius, n=n)

    df = pd.DataFrame({
        'a': list(X_a),
        'x_u': list(X_u)
        })

    return df


# Training and testing data
n_train = 1000
n_test = 100
theta_train = sample(theta_bar, radius, n=n_train)
theta_test = sample(theta_bar, radius, n=n_test)

## Train predictor (Pytorch)

In [7]:
# Dictionary of different parameters.
# The cross validation will try all of the possible
# combinations
params = {
    'learning_rate': [0.001, 0.01],
    'batch_size': [132],
    'n_epochs': [10]
}
m.train(theta_train, learner=mlopt.PYTORCH, params=params)

INFO:root:Use new data
INFO:root:Compute tight constraints for training set (n_jobs = 4)
100%|██████████| 1000/1000 [00:15<00:00, 65.61it/s]
INFO:root:Encoding strategies
INFO:root:Getting unique set of strategies
INFO:root:Found 43 unique strategies
INFO:root:Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)
100%|██████████| 43/43 [00:00<00:00, 87.76it/s]
INFO:root:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 7169.08it/s]
INFO:root:Split dataset in 900 training and 100 validation
INFO:root:Train Neural Network with 2 sets of parameters, 20 inputs, 43 outputs
INFO:root:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 132, 'n_epochs': 10, 'n_hidden': 31}
INFO:root:Epoch 1/10
100%|██████████| 7/7 [00:00<00:00, 24.89it/s, loss=3.759]
INFO:root:- Train metrics: accuracy: 0.008 ; loss: 3.790
INFO:root:- Eval metrics: accuracy: 0.40

## Benchmark on testing dataset

In [8]:
results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

INFO:root:Performance evaluation
INFO:root:Compute tight constraints for test set (n_jobs = 1)
  0%|          | 0/100 [00:00<?, ?it/s]

Academic license - for non-commercial use only


INFO:gurobipy:Academic license - for non-commercial use only
Converting dataframe to array: 100%|██████████| 100/100 [00:00<00:00, 5753.66it/s]
INFO:root:Predict tight constraints for test set
100%|██████████| 100/100 [00:04<00:00, 22.48it/s]


Accuracy: 62.00 


## Save training data

In [9]:
m.save_training_data("training_data.pkl", delete_existing=True)

## Create new solver and train passing loaded data

In [10]:
m = mlopt.Optimizer(cp.Minimize(cost), constraints)
m.load_training_data("training_data.pkl")
m.train(learner=mlopt.PYTORCH, params=params)  # Train after loading samples

results = m.performance(theta_test)
print("Accuracy: %.2f " % results[0]['accuracy'])

INFO:root:Caching KKT solver factors for each strategy (it works only for QP-representable problems with parameters only in constraints RHS)
100%|██████████| 43/43 [00:00<00:00, 58.59it/s]
INFO:root:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 6214.31it/s]
INFO:root:Split dataset in 900 training and 100 validation
INFO:root:Train Neural Network with 2 sets of parameters, 20 inputs, 43 outputs
INFO:root:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 132, 'n_epochs': 10, 'n_hidden': 31}
INFO:root:Epoch 1/10
100%|██████████| 7/7 [00:00<00:00, 120.19it/s, loss=3.698]
INFO:root:- Train metrics: accuracy: 0.205 ; loss: 3.713
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.684
INFO:root:Epoch 2/10
100%|██████████| 7/7 [00:00<00:00, 172.83it/s, loss=3.630]
INFO:root:- Train metrics: accuracy: 0.394 ; loss: 3.661
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.593
INFO:root:Epoch 3/10
100%|██████████| 7/7

Accuracy: 62.00 


## Predict single point

In [11]:
# Predict single point
theta = theta_test.iloc[0]
result_single_point = m.solve(theta)

Converting dataframe to array: 100%|██████████| 1/1 [00:00<00:00, 841.05it/s]
INFO:root:Predict optimal solution
100%|██████████| 1/1 [00:00<00:00,  9.97it/s]


## Learn directly from points (talk directly to pytorch)

In [13]:
y = m.y_train
X = m.X_train
learner = PyTorchNeuralNet(n_input=n_features(X),
                           n_classes=len(np.unique(y)),
                           n_best=3,
                           params=params)
# Train learner
learner.train(pandas2array(X), y)

# Predict
X_pred = X.iloc[0]
y_pred = learner.predict(pandas2array(X_pred))  # n_best most likely classes

INFO:root:Using CPU with Pytorch
Converting dataframe to array: 100%|██████████| 1000/1000 [00:00<00:00, 4045.27it/s]
INFO:root:Split dataset in 900 training and 100 validation
INFO:root:Train Neural Network with 2 sets of parameters, 20 inputs, 43 outputs
INFO:root:Learning Neural Network with parameters: {'learning_rate': 0.001, 'batch_size': 132, 'n_epochs': 10, 'n_hidden': 31}
INFO:root:Epoch 1/10
100%|██████████| 7/7 [00:00<00:00, 120.63it/s, loss=3.698]
INFO:root:- Train metrics: accuracy: 0.205 ; loss: 3.713
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.684
INFO:root:Epoch 2/10
100%|██████████| 7/7 [00:00<00:00, 182.80it/s, loss=3.630]
INFO:root:- Train metrics: accuracy: 0.394 ; loss: 3.661
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.593
INFO:root:Epoch 3/10
100%|██████████| 7/7 [00:00<00:00, 92.05it/s, loss=3.522] 
INFO:root:- Train metrics: accuracy: 0.394 ; loss: 3.570
INFO:root:- Eval metrics: accuracy: 0.320 ; loss: 3.462
INFO:root:Epoch 4/10
100%|██████████|